In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))

In [2]:
install.packages('vegan')

also installing the dependency ‘permute’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.3. Cluster Analysis/Output'
cur_date = '071925'

library(readxl)
library(tidyverse)
library(reshape2)
library(factoextra)
library(FactoMineR)
library(gridExtra)
library(ggrepel)
library(RColorBrewer)
library(gridtext)
library(vegan)

# reading in files
full_df = data.frame(read_excel("Input/Processed_Data_070825.xlsx")) %>%
    # creating sample ids
    unite("SampleID", c("HAWC_ID", "Sample_Number"), remove = FALSE)

In [5]:
head(full_df)

,Sample,SampleID,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Cr,Chromium,NA
2,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Cu,Copper,2.744161
3,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Pb,Lead,2.584963
4,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Ti,Titanium,5.087463
5,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Zn,Zinc,4.523562
6,WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,K,Potassium,NA


This analysis will use MCA to explore if chemical and metal concentrations of control, WS, and WF samples are similar.

In [6]:
# will run MCA on each class, so the df will be split
split_df = full_df %>%
    # removing outliers
    group_by(Metric) %>%
    group_split

vol_df = split_df[[1]]
weight_df = split_df[[2]]

head(vol_df)

Sample,SampleID,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Cr,Chromium,NA
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Cu,Copper,2.744161
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Pb,Lead,2.584963
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Ti,Titanium,5.087463
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,Zn,Zinc,4.523562
WF,1289909_1,1289909,Trieu et al. 2024,1,Metal,Volume,K,Potassium,NA


In [7]:
`%notin%` <- Negate(`%in%`) 

wider_vol_df = vol_df %>%
    select(-c("Name", "Class")) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Value") %>%
    column_to_rownames(var = "SampleID")

wider_weight_df = weight_df %>%
    select(-c("Name", "Class")) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Value") %>%
    column_to_rownames(var = "SampleID")

head(wider_vol_df)

,Sample,HAWC_ID,Study,Sample_Number,Metric,Cr,Cu,Pb,Ti,Zn,K,Ca,Fe,Mn,Ni,Sr
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1289909_1,WF,1289909,Trieu et al. 2024,1,Volume,NA,2.744161,2.584963,5.087463,4.523562,NA,9.390169,8.233620,2.560715,1.000000,1.807355
885041_1,WS,885041,Ihantola et al. 2020,1,Volume,5.865424,9.820179,6.930737,4.940167,13.538795,16.91999,11.761136,10.561288,7.266787,4.822730,4.078951
885041_2,WS,885041,Ihantola et al. 2020,2,Volume,5.523562,8.640245,6.061776,5.281698,12.746304,16.19654,11.948002,9.383704,6.832890,4.655352,3.264536
993956_1,WS,993956,Ihantola et al. 2022,1,Volume,6.285402,7.577429,6.584963,4.954196,14.053332,15.87270,11.451726,10.452241,8.134426,5.584963,4.392317
1263480_1,WS,1263480,Burnet et al. 1990,1,Volume,NA,13.550867,12.425478,NA,18.116806,21.61807,18.892083,15.383671,13.550867,NA,12.773345
1263480_2,WS,1263480,Burnet et al. 1990,2,Volume,NA,13.340963,NA,11.358651,20.306748,20.07464,18.694559,14.587895,13.074476,10.426265,12.720886


# Multiple Correspondence Analysis (MCA)

In [17]:
data(poison)
poison.active <- poison[1:55, 5:15]

In [18]:
poison.active

,Nausea,Vomiting,Abdominals,Fever,Diarrhae,Potato,Fish,Mayo,Courgette,Cheese,Icecream
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,Nausea_y,Vomit_n,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y
2,Nausea_n,Vomit_n,Abdo_n,Fever_n,Diarrhea_n,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_n,Icecream_y
3,Nausea_n,Vomit_y,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y
4,Nausea_n,Vomit_n,Abdo_n,Fever_n,Diarrhea_n,Potato_y,Fish_y,Mayo_n,Courg_y,Cheese_y,Icecream_y
5,Nausea_n,Vomit_y,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y
6,Nausea_n,Vomit_n,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_n,Mayo_y,Courg_y,Cheese_y,Icecream_y
7,Nausea_n,Vomit_y,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y
8,Nausea_y,Vomit_y,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y
9,Nausea_y,Vomit_n,Abdo_y,Fever_y,Diarrhea_y,Potato_y,Fish_y,Mayo_y,Courg_y,Cheese_y,Icecream_y


In [13]:
# do we not need to worry about scaling and centering data for mca?
# running MCA
vol_mca = MCA(wider_vol_df[,c(6:ncol(wider_vol_df))])
weight_mca = MCA(wider_weight_df[,c(6:ncol(wider_weight_df))])

# looking at a scree plot to see how much of the variance was captured in the first 2 eigenvectors
options(repr.plot.width = 10, repr.plot.height = 5) #changing size
fviz_eig(vol_mca, addlabels = FALSE)
fviz_eig(weight_mca, addlabels = FALSE)

Warning message:
“ggrepel: 6 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“Removed 11 rows containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 11 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 21 rows containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 21 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”


In [16]:
vol_mca$eig

,eigenvalue,percentage of variance,cumulative percentage of variance
dim 1,1.0000000,14.285714,14.28571
dim 2,1.0000000,14.285714,28.57143
dim 3,1.0000000,14.285714,42.85714
dim 4,0.9144703,13.063861,55.92100
dim 5,0.8626415,12.323450,68.24445
dim 6,0.8360625,11.943750,80.18820
dim 7,0.7272727,10.389610,90.57781
dim 8,0.6595530,9.422186,100.00000
